In [6]:
# Install the necessary libraries (uncomment if not already installed in Colab)
# !pip install yfinance pandas matplotlib ipywidgets

import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import time

# 1. Define function to get current stock price
def get_stock_price(symbol):
    try:
        stock = yf.Ticker(symbol)
        current_price = stock.history(period="1d")['Close'][0]
        return current_price
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None  # Return None if there's an error fetching the data

# 2. Function to track portfolio value and profit/loss
def track_portfolio(stocks, shares, purchase_prices):
    # Create portfolio DataFrame
    portfolio = {'Stock': stocks, 'Shares': shares, 'Purchase Price': purchase_prices}
    portfolio_df = pd.DataFrame(portfolio)

    # Get current stock prices
    portfolio_df['Current Price'] = portfolio_df['Stock'].apply(get_stock_price)

    # Filter out any rows where data is missing (invalid stock symbols)
    portfolio_df = portfolio_df[portfolio_df['Current Price'].notna()]

    if portfolio_df.empty:
        print("Error: No valid stock data found.")
        return

    # Calculate current value, total cost, and profit/loss
    portfolio_df['Current Value'] = portfolio_df['Current Price'] * portfolio_df['Shares']
    portfolio_df['Total Cost'] = portfolio_df['Purchase Price'] * portfolio_df['Shares']
    portfolio_df['Profit/Loss'] = portfolio_df['Current Value'] - portfolio_df['Total Cost']

    # Calculate total portfolio value and total profit/loss
    total_value = portfolio_df['Current Value'].sum()
    total_profit_loss = portfolio_df['Profit/Loss'].sum()

    # Output results
    print("\nStock Portfolio Overview:")
    print(portfolio_df)
    print(f"\nTotal Portfolio Value: ${total_value:.2f}")
    print(f"Total Profit/Loss: ${total_profit_loss:.2f}")

    # 1. Visualize the stock price of the first stock (example: AAPL) as a graph
    stock_data = yf.Ticker(portfolio_df['Stock'].iloc[0])
    stock_history = stock_data.history(period="1mo")  # Fetch data for the last month

    # Plotting stock price over time
    plt.figure(figsize=(10, 6))
    stock_history['Close'].plot(title=f"{portfolio_df['Stock'].iloc[0]} Stock Price Over Time", color='blue')
    plt.xlabel("Date")
    plt.ylabel("Price ($)")
    plt.grid(True)
    plt.show()

    # 2. Create a bar chart to visualize portfolio data: Current Value, Total Cost, and Profit/Loss
    portfolio_df.set_index('Stock', inplace=True)

    # Plotting a bar chart for portfolio data (Current Value, Purchase Price, and Profit/Loss)
    fig, axes = plt.subplots(1, 3, figsize=(15, 6))

    # Current Value Bar Chart
    portfolio_df['Current Value'].plot(kind='bar', ax=axes[0], color='skyblue', title='Current Value')
    axes[0].set_ylabel("Value ($)")
    axes[0].set_xlabel("Stock")
    axes[0].tick_params(axis='x', rotation=45)

    # Total Cost Bar Chart
    portfolio_df['Total Cost'].plot(kind='bar', ax=axes[1], color='lightgreen', title='Total Cost')
    axes[1].set_ylabel("Cost ($)")
    axes[1].set_xlabel("Stock")
    axes[1].tick_params(axis='x', rotation=45)

    # Profit/Loss Bar Chart
    portfolio_df['Profit/Loss'].plot(kind='bar', ax=axes[2], color='salmon', title='Profit/Loss')
    axes[2].set_ylabel("Profit/Loss ($)")
    axes[2].set_xlabel("Stock")
    axes[2].tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.show()

# 3. Define function to prompt user input with background color changes
def user_input():
    # Create widgets for user input with improved styling and background color
    stock_input = widgets.Text(
        description='Stock Symbol(s):',
        placeholder='e.g., AAPL, GOOGL',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px', background_color='lightyellow')  # Background color for stock input
    )
    shares_input = widgets.Text(
        description='Shares:',
        placeholder='e.g., 10, 5',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px')
    )
    price_input = widgets.Text(
        description='Purchase Price(s):',
        placeholder='e.g., 150.0, 1000.0',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px', background_color='lightblue')  # Background color for price input
    )

    # Create a progress bar widget
    progress_bar = widgets.FloatProgress(
        value=0.0,
        min=0.0,
        max=1.0,
        description='Processing: ',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px')
    )

    # Create a button to submit the portfolio information
    submit_button = widgets.Button(description="Track Portfolio", layout=widgets.Layout(width='200px'))
    submit_button.style.button_color = 'lightgreen'

    # Output area to display portfolio details
    output = widgets.Output()

    # Function to handle button click event
    def on_button_click(b):
        # Show the progress bar while processing
        progress_bar.value = 0.1
        clear_output(wait=True)
        display(stock_input, shares_input, price_input, submit_button, progress_bar, output)

        # Delay to simulate processing (simulate loading with incremented progress bar)
        time.sleep(1)
        progress_bar.value = 0.3
        time.sleep(1)
        progress_bar.value = 0.5
        time.sleep(1)
        progress_bar.value = 0.7
        time.sleep(1)
        progress_bar.value = 0.9
        time.sleep(1)

        # Get user inputs
        stock = stock_input.value.upper().split(',')  # Split stock symbols by commas
        shares = list(map(int, shares_input.value.split(',')))  # Convert shares input into integers
        purchase_prices = list(map(float, price_input.value.split(',')))  # Convert prices into floats

        # Check if the lengths of the lists match
        if len(stock) != len(shares) or len(stock) != len(purchase_prices):
            print("Error: The number of stocks, shares, and purchase prices must match.")
            progress_bar.value = 0.0
            return

        # Call track portfolio function
        with output:
            track_portfolio(stock, shares, purchase_prices)

        # Hide the progress bar after processing
        progress_bar.value = 1.0

    # Bind button click to function
    submit_button.on_click(on_button_click)

    # Display widgets with improved layout
    display(stock_input, shares_input, price_input, submit_button, progress_bar, output)

# 4. Run the user input function
user_input()


Text(value='AAPL', description='Stock Symbol(s):', layout=Layout(width='400px'), placeholder='e.g., AAPL, GOOG…

Text(value='20', description='Shares:', layout=Layout(width='400px'), placeholder='e.g., 10, 5', style=Descrip…

Text(value='1900', description='Purchase Price(s):', layout=Layout(width='400px'), placeholder='e.g., 150.0, 1…

Button(description='Track Portfolio', layout=Layout(width='200px'), style=ButtonStyle(button_color='lightgreen…

FloatProgress(value=0.1, description='Processing: ', layout=Layout(width='400px'), max=1.0, style=ProgressStyl…

Output()